In [ ]:
import numpy as np
from ase.build import bulk
from ase.calculators.emt import EMT
from phono3py import Phono3py

from vibes.structure.convert import to_phonopy_atoms, to_Atoms

In [ ]:
atoms = bulk('Al', cubic=True)

smatrix = np.diag([2, 2, 2])
q_mesh = 3 * [5, ]

In [ ]:
calc = EMT()

In [ ]:
phonon3 = Phono3py(
    to_phonopy_atoms(atoms), supercell_matrix=smatrix, log_level=2
)

In [ ]:
phonon3.generate_displacements(cutoff_pair_distance=5)

In [ ]:
primitive = to_Atoms(phonon3.primitive)
supercell = to_Atoms(phonon3.supercell)

primitive

In [ ]:
scs = phonon3.supercells_with_displacements

In [ ]:
force_sets = []
for pa in scs:
    if pa is None:
        force_sets.append(np.zeros([len(supercell), 3]))
        continue
    a = to_Atoms(pa)
    a.calc = EMT()
    force_sets.append(a.get_forces())

In [ ]:
phonon3.forces = np.array(force_sets)

In [ ]:
phonon3.produce_fc2()
phonon3.produce_fc3()

In [ ]:
phonon3.mesh_numbers = [6, 6, 6]
phonon3.init_phph_interaction()
phonon3.run_thermal_conductivity(write_kappa=True)